In [1]:
#import sklearn to cluster

import numpy as np
import pandas as pd
import time as tm
import math # to calculate distance between servers

from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA # use pca reduce dims
from mpl_toolkits.mplot3d import Axes3D # plot 3d figure
from s_dbw import S_Dbw # evaluate clustering

from matplotlib import pyplot

from sklearn.cluster import MeanShift # a clustering algorithm decide the n_cluster automatically
from itertools import cycle # for mean-shift plot

In [2]:
# Data Processing - read data from .csv file

data_loc = 'us'
file = f'../dataset/{data_loc}.csv'

df = pd.read_csv(file)
df = pd.DataFrame(df)

time_start = tm.time()

ip_time = dict()
special_chr = ["{", "}", "'", ":", " "]

for i in range(len(df)):
    raw_points = df['transactionList'][i]
    points = ''.join(e for e in raw_points if e not in special_chr)
    points = points.split(",")
    for j in range(len(points)):
        time = points[j][:17]
        ip = points[j][17:]
        if ip not in ip_time:
            ip_time[ip] = [time]
        else:
            ip_time[ip].append(time)

time_end = tm.time()
runtime = time_end - time_start
print(f'runtime: {runtime} second')

total_num = len(ip_time)
print(f'num of total server: {total_num}')

print(f'Data Size of "{data_loc}": {len(df)}')

C:\Users\smart\AppData\Local\Temp\ipykernel_16624\901789945.py:6: DtypeWarning: Columns (1,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


runtime: 47.72029781341553 second
num of total server: 619
Data Size of "us": 284249


In [3]:
# select specific period from the dataset, "ip_time"

def get_ip_time(ip_data=ip_time, period='2021', period_ip_time=None): # period_ip_time: use to accumulate date
    if period_ip_time is None:
        period_ip_time = dict()
    if type(period) == str:
        len_p = len(period)
        for ip in ip_data:
            for time in ip_data[ip]:
                if time[0:len_p] == period:
                    if ip not in period_ip_time:
                        period_ip_time[ip] = [time]
                    else:
                        period_ip_time[ip] += [time]
    elif type(period) == list: #[time_1, time_2, ...]
        period_ip_time = get_ip_time(period=period[0])
        for time_i in period[1:]:
            period_ip_time = get_ip_time(period=time_i, period_ip_time=period_ip_time)
    return period_ip_time

# get 2021 data - US-All
data_us_all = get_ip_time()

# get US-1 data - 2021-05-07~2021-05-16 (for us dataset)
us_1 = ['2021-05-07', '2021-05-08', '2021-05-09', '2021-05-10', '2021-05-11',
            '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-15', '2021-05-16']
data_us_1 = get_ip_time(period=us_1)

# check the data
for ip in data_us_1:
    print(f'ip: {ip}')
    print(data_us_1[ip][:5])
    break

ip: 52.223.228.27
['2021-05-07T000111', '2021-05-07T002416', '2021-05-07T003212', '2021-05-07T002644', '2021-05-07T003952']


In [ ]:
# setup hour axis for below chart
hour_axis = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
             '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [ ]:
# Table 5.1: Estimation model - Error Rate

print('Table 5.1: Estimation model - Error Rate')